# Pre-processing Tutorial

## Introduction

*   Removal of rows with missing values
*   Replacement of missing values with default values
*   Dectection of entity resolution problems
*   Simple entity resolution
*   Removal of unpaired entities
*   Replacement of categorical variables with indicators
*   Identification of candidate indices
*   Data exportation

## Setup

Run the cell below to import necessary modules.

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive


To prepare the datasets for ingestion into the database, we need to:
1. Clean missing / misentered values
2. Detect and solve entity resolution problems
3. Replace categorical variables with numeric indicators for efficiency
4. Export the data

Before doing any of this, we need to import the datasets to _pandas_ `DataFrames`. We'll follow the data importation procedure outlined in the [EDA tutorial](https://drive.google.com/open?id=1Cy3izai9zLQYTCTQF9IwkcuLmNArcKZO). Add the datasets to your Google Drive using the links above, then mount your Drive to the notebook by running the cell below.

In [2]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


Finally, run the cell below to load the census data into a `DataFrame` named `census` and the life expectancy data into a `DataFrame` named `le`.

In [3]:
s_bundesliga = pd.read_csv("/content/drive/My Drive/trades_data/1-bundesliga.csv")
s_champ = pd.read_csv("/content/drive/My Drive/trades_data/championship.csv")
s_eredivisie = pd.read_csv("/content/drive/My Drive/trades_data/eredivisie.csv")
s_liganos = pd.read_csv("/content/drive/My Drive/trades_data/liga-nos.csv")
s_ligue1 = pd.read_csv("/content/drive/My Drive/trades_data/ligue-1.csv")
s_premierleague = pd.read_csv("/content/drive/My Drive/trades_data/premier-league.csv")
s_premierliga = pd.read_csv("/content/drive/My Drive/trades_data/premier-liga.csv")
s_primeradiv = pd.read_csv("/content/drive/My Drive/trades_data/primera-division.csv")
s_seriea = pd.read_csv("/content/drive/My Drive/trades_data/serie-a.csv")

t_appearances = pd.read_csv("/content/drive/My Drive/transfermarkt_data/appearances.csv")
t_clubgames = pd.read_csv("/content/drive/My Drive/transfermarkt_data/club_games.csv")
t_clubs = pd.read_csv("/content/drive/My Drive/transfermarkt_data/clubs.csv")
t_comps = pd.read_csv("/content/drive/My Drive/transfermarkt_data/competitions.csv")
t_events = pd.read_csv("/content/drive/My Drive/transfermarkt_data/game_events.csv")
t_games = pd.read_csv("/content/drive/My Drive/transfermarkt_data/games.csv")
t_playerval = pd.read_csv("/content/drive/My Drive/transfermarkt_data/player_valuations.csv")
t_players = pd.read_csv("/content/drive/My Drive/transfermarkt_data/players.csv")


## Clean Missing Values

1. Clubs.csv: remove rows with no club name, create new column called “transfer_euros” with cleaned net_transfer_record data (remove currency symbol, replace abbreviations with appropriate multiple)
2. Players.csv: for the players who are known by a single name (empty first name), replace the empty first name with the entry in last name and replace the last name entry with “”
3. all trades .csv files replace ‘NA’ in fee_cleaned with -1, since there is no negative fee values in the column and we have a column referring to if the fee transfer amount is in/out

In [4]:
def convert_transfer(record):
    if record == "+-0":
        return 0
    elif 'm' in record:
        return float(record.replace('€', '').replace('m', '')) * 1e6
    elif 'k' in record:
        return float(record.replace('€', '').replace('k', '')) * 1e3
    else:
        return float(record.replace('€', ''))

t_clubs = t_clubs[t_clubs["name"].notna()] #remove rows without country
t_clubs = t_clubs.drop(columns =['total_market_value', 'coach_name'])

t_clubs['transfer_euros'] = t_clubs['net_transfer_record'].apply(convert_transfer)
t_clubs

,club_id,club_code,name,domestic_competition_id,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,last_season,url,transfer_euros
2,2239,us-cremonese,US Cremonese,IT1,26,26.0,12,46.2,7,Giovanni Zini,16003,€-33.80m,2022,https://www.transfermarkt.co.uk/us-cremonese/s...,-33800000.0
3,2423,cd-santa-clara,CD Santa Clara,PO1,31,25.8,20,64.5,2,Estádio de São Miguel,15000,+€5.85m,2022,https://www.transfermarkt.co.uk/cd-santa-clara...,5850000.0
4,24245,umraniyespor,Ümraniyespor,TR1,24,28.4,10,41.7,3,Ümraniye Belediyesi Şehir Stadı,3513,+-0,2022,https://www.transfermarkt.co.uk/umraniyespor/s...,0.0
5,2553,kilmarnock-fc,Kilmarnock FC,SC1,27,25.7,16,59.3,1,"BBSP Stadium, Rugby Park",18128,+-0,2022,https://www.transfermarkt.co.uk/kilmarnock-fc/...,0.0
7,2671,pas-giannina,PAS Giannina,GR1,31,25.4,14,45.2,0,Stadio Zosimades,7652,+€2.68m,2022,https://www.transfermarkt.co.uk/pas-giannina/s...,2680000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,660,shakhtar-donetsk,Shakhtar Donetsk,UKR1,31,24.7,6,19.4,14,Arena Lviv,34915,+€102.90m,2022,https://www.transfermarkt.co.uk/shakhtar-donet...,102900000.0
405,749,fc-empoli,FC Empoli,IT1,30,25.2,14,46.7,9,Carlo Castellani,19847,+€16.51m,2022,https://www.transfermarkt.co.uk/fc-empoli/star...,16510000.0
407,82,vfl-wolfsburg,VfL Wolfsburg,L1,25,24.6,15,60.0,10,Volkswagen Arena,30000,€-12.05m,2022,https://www.transfermarkt.co.uk/vfl-wolfsburg/...,-12050000.0
408,940,celta-vigo,Celta de Vigo,ES1,23,26.3,10,43.5,6,Abanca Balaídos,29000,€-5.15m,2022,https://www.transfermarkt.co.uk/celta-vigo/sta...,-5150000.0


In [5]:
t_players['single'] = (t_players['first_name'].isnull()) & ~(t_players['last_name'].isnull())

t_players['first_name'] = t_players.apply(lambda row: row['last_name'] if row['single'] else row['first_name'], axis=1)
t_players['last_name'] = t_players.apply(lambda row: '' if row['single'] else row['last_name'], axis=1)

t_players = t_players[~(t_players['first_name'].isnull() & t_players['last_name'].isnull())]
t_players = t_players.drop(columns =['single'])
t_players

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name
0,597,Aleksandr,Hleb,Aleksandr Hleb,2016,2696,aleksandr-hleb,UdSSR,Minsk,Belarus,...,NaN,NaN,NaN,19000000.0,NaN,fair-sport GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aleksandr-hleb...,RU1,Krylya Sovetov Samara
1,1428,Mike,Hanke,Mike Hanke,2013,60,mike-hanke,Germany,Hamm,Germany,...,right,185.0,NaN,5000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/mike-hanke/pro...,L1,SC Freiburg
2,1560,Youssef,El Akchaoui,Youssef El Akchaoui,2012,306,youssef-el-akchaoui,Netherlands,Dordrecht,Morocco,...,NaN,NaN,NaN,1750000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/youssef-el-akc...,NL1,SC Heerenveen
3,1587,Mario,Eggimann,Mario Eggimann,2012,42,mario-eggimann,Switzerland,Brugg,Switzerland,...,right,190.0,NaN,3000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/mario-eggimann...,L1,NaN
4,1726,Heiko,Westermann,Heiko Westermann,2016,610,heiko-westermann,Germany,Alzenau,Germany,...,right,190.0,NaN,9000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/heiko-westerma...,NL1,Ajax Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28454,199337,Rifat,Erdem,Rifat Erdem,2013,2293,rifat-erdem,Turkey,Konya,Turkey,...,right,NaN,NaN,100000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/rifat-erdem/pr...,TR1,Konyaspor
28455,209569,Mücahit,Atalay,Mücahit Atalay,2020,2293,mucahit-atalay,Turkey,Trabzon,Turkey,...,right,186.0,NaN,250000.0,2023-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/mucahit-atalay...,TR1,Konyaspor
28456,221118,Patrick Friday,Eze,Patrick Friday Eze,2017,2293,patrick-friday-eze,Nigeria,Kaduna,Nigeria,...,right,187.0,200000.0,775000.0,2023-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/patrick-friday...,TR1,Konyaspor
28457,273053,Burak,Saban,Burak Saban,2018,2293,burak-saban,Netherlands,Hoorn,Netherlands,...,both,177.0,NaN,200000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/burak-saban/pr...,TR1,Konyaspor


In [6]:
s_bundesliga['fee_cleaned'].fillna(-1, inplace=True)
s_champ['fee_cleaned'].fillna(-1, inplace=True)
s_eredivisie['fee_cleaned'].fillna(-1, inplace=True)
s_liganos['fee_cleaned'].fillna(-1, inplace=True)
s_ligue1['fee_cleaned'].fillna(-1, inplace=True)
s_premierleague['fee_cleaned'].fillna(-1, inplace=True)
s_premierliga['fee_cleaned'].fillna(-1, inplace=True)
s_primeradiv['fee_cleaned'].fillna(-1, inplace=True)
s_seriea['fee_cleaned'].fillna(-1, inplace=True)

s_bundesliga

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,1.FC Saarbrücken,Thomas Stratos,25.0,Defensive Midfield,Hamburger SV,€150Th.,in,Summer,0.15,1 Bundesliga,1992,1992/1993
1,1.FC Saarbrücken,Matthias Lust,22.0,Left Midfield,Waldh. Mannheim,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993
2,1.FC Saarbrücken,Henning Bürger,22.0,Left Midfield,FC Schalke 04,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993
3,1.FC Saarbrücken,Michael Hennig,20.0,midfield,FC Berlin,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993
4,1.FC Saarbrücken,Malik Badji,24.0,defence,CS Sedan,?,in,Summer,-1.00,1 Bundesliga,1992,1992/1993
...,...,...,...,...,...,...,...,...,...,...,...,...
13488,SpVgg Greuther Fürth,Mert Yusuf Torlak,19.0,Attacking Midfield,Gr. Fürth II,-,out,Winter,-1.00,1 Bundesliga,2021,2021/2022
13489,SpVgg Greuther Fürth,Elias Kratzer,21.0,Right-Back,Gr. Fürth II,-,out,Winter,-1.00,1 Bundesliga,2021,2021/2022
13490,SpVgg Greuther Fürth,Adrian Fein,22.0,Defensive Midfield,Bayern Munich,"End of loanJan 30, 2022",out,Winter,-1.00,1 Bundesliga,2021,2021/2022
13491,SpVgg Greuther Fürth,Justin Hoogma,23.0,Centre-Back,TSG Hoffenheim,"End of loanJan 23, 2022",out,Winter,-1.00,1 Bundesliga,2021,2021/2022


## Entity Resolution

Now that we've handled missing values in both datasets, we turn our attention to performing entity resolution on the entities common to both. In this case, those common entities are countries.

To perform entity resolution, we will:
1. Determine whether both datasets use the same names to refer to all countries
2. Edit the names in one dataset to match the other, if necessary

In the general case, you may also need to detect when datasets refer to different entities using the same name/ID and disambiguate these references. This may happen when handling datasets that contain multiple people with the same name, for example. But we don't need to worry about it here because the names of countries are well-known and distict.


### Detect Inconsistent Names
Let's compile a list of all country names in both datasets, then inspect it for repetitions.

First, we extract the unique names of countries from both datasets.

In [7]:
#find club names datasets

bundsliga_names = s_bundesliga["club_name"].unique()
champ_names = s_champ["club_name"].unique()
eredivisie_names = s_eredivisie["club_name"].unique()
liganos_names = s_liganos["club_name"].unique()
ligue1_names = s_ligue1["club_name"].unique()
premierleague_names = s_premierleague["club_name"].unique()
premierliga_names = s_premierliga["club_name"].unique()
primeradiv_names = s_primeradiv["club_name"].unique()
seria_names = s_seriea["club_name"].unique()

s_names = set(bundsliga_names.tolist() + champ_names.tolist() + eredivisie_names.tolist() + liganos_names.tolist() + ligue1_names.tolist() + premierleague_names.tolist() + premierliga_names.tolist() + primeradiv_names.tolist() + seria_names.tolist())

t_names = set(t_clubs["name"].unique())

Now, we combine these into one set.

In [8]:
#find set difference in club names

result_set = s_names.symmetric_difference(t_names)
result_set


{'1. FC Köln',
 '1.FC Kaiserslautern',
 '1.FC Köln',
 '1.FC Nuremberg',
 '1.FC Saarbrücken',
 'AC Cesena',
 'AC Fiorentina',
 'AC Horsens',
 'AC Le Havre',
 'AC Monza',
 'AC Parma',
 'AC Perugia',
 'AC Reggiana',
 'AC Siena',
 'AC Venezia 1907',
 'ADO Den Haag',
 'AEK Athens',
 'APO Levadiakos',
 'AS Bari',
 'AS Cannes',
 'AS Livorno',
 'AS Nancy-Lorraine',
 'AS Saint-Étienne',
 'Aalborg BK',
 'Aarhus GF',
 'Aberdeen FC',
 'Académica Coimbra',
 'Adana Demirspor',
 'Alania Vladikavkaz',
 'Alanyaspor',
 'Albacete Balompié',
 'Alemannia Aachen',
 'Amiens SC',
 'Amkar Perm',
 'Ancona Calcio',
 'Antalyaspor',
 'Anzhi Makhachkala ( -2022)',
 'Aris Thessaloniki',
 'Arminia Bielefeld',
 'Arsenal Tula',
 'Ascoli Calcio 1898',
 'Association Troyes Aube Champagne',
 'Asteras Tripolis',
 'Athlétic Club Arlésien',
 'Atromitos Athen',
 'Baltika Kaliningrad',
 'Barnsley FC',
 'Basaksehir FK',
 'Bayer 05 Uerdingen',
 'Belenenses SAD',
 'Benevento Calcio',
 'Besiktas JK',
 'Birmingham City',
 'Blackbur

In [9]:
import re

def revise_string(s):
    #remove hypens
    s = s.replace('-', ' ')
    #remove anything in parenthesis, including the parentheses
    s = re.sub(r'\([^)]*\)', '', s)
    #remove all numbers
    s = re.sub(r'\d+', '', s)
    #remove periods
    s = s.replace('.', '')
    #remove double spaces
    s = re.sub(r'\s+', ' ', s)
    #remove extra spaces at the front and end
    s = s.strip()
    return s

revised_s_names = {revise_string(s) for s in s_names}
revised_t_names = {revise_string(s) for s in t_names}
result_set2 = revised_s_names.symmetric_difference(revised_t_names)
result_set2

{'AC Cesena',
 'AC Fiorentina',
 'AC Horsens',
 'AC Le Havre',
 'AC Monza',
 'AC Parma',
 'AC Perugia',
 'AC Reggiana',
 'AC Siena',
 'AC Venezia',
 'ADO Den Haag',
 'AEK Athens',
 'APO Levadiakos',
 'AS Bari',
 'AS Cannes',
 'AS Livorno',
 'AS Nancy Lorraine',
 'AS Saint Étienne',
 'Aalborg BK',
 'Aarhus GF',
 'Aberdeen FC',
 'Académica Coimbra',
 'Adana Demirspor',
 'Alania Vladikavkaz',
 'Alanyaspor',
 'Albacete Balompié',
 'Alemannia Aachen',
 'Amiens SC',
 'Amkar Perm',
 'Ancona Calcio',
 'Antalyaspor',
 'Anzhi Makhachkala',
 'Aris Thessaloniki',
 'Arminia Bielefeld',
 'Arsenal Tula',
 'Ascoli Calcio',
 'Association Troyes Aube Champagne',
 'Asteras Tripolis',
 'Athlétic Club Arlésien',
 'Atromitos Athen',
 'Baltika Kaliningrad',
 'Barnsley FC',
 'Basaksehir FK',
 'Bayer Uerdingen',
 'Belenenses SAD',
 'Benevento Calcio',
 'Besiktas JK',
 'Birmingham City',
 'Blackburn Rovers',
 'Blackpool FC',
 'Bolton Wanderers',
 'Bradford City',
 'Brescia Calcio',
 'Bristol City',
 'Bröndby IF

In [10]:
t_clubs['name'] = t_clubs['name'].apply(revise_string)
s_bundesliga['club_name'] = s_bundesliga['club_name'].apply(revise_string)
s_champ['club_name'] = s_champ['club_name'].apply(revise_string)
s_eredivisie['club_name'] = s_eredivisie['club_name'].apply(revise_string)
s_liganos['club_name'] = s_liganos['club_name'].apply(revise_string)
s_ligue1['club_name'] = s_ligue1['club_name'].apply(revise_string)
s_premierleague['club_name'] = s_premierleague['club_name'].apply(revise_string)
s_premierliga['club_name'] = s_premierliga['club_name'].apply(revise_string)
s_primeradiv['club_name'] = s_primeradiv['club_name'].apply(revise_string)
s_seriea['club_name'] = s_seriea['club_name'].apply(revise_string)

The output contains clubs with multiple names:

*   'Saturn REN TV Ramenskoe' and 'Saturn Ramenskoe'
*   'Spartak Alania Vladikavkaz' and 'Spartak Vladikavkaz'
*   'Genoa' and 'Genoa CFC'
*   'FC Nizhniy Novgorod' and 'FC Pari Nizhniy Novgorod'

### Resolve Inconsistent Names

We first create a `Series` of the countries with multiple names, where of each element is the name we'll purge, and the value is name we'll keep.

In [11]:
name_map = {
    'Saturn Ramenskoe' : 'Saturn REN TV Ramenskoe',
    'Spartak Vladikavkaz' : 'Spartak Alania Vladikavkaz',
    'Genoa' : 'Genoa CFC',
    'FC Nizhniy Novgorod' : 'FC Pari Nizhniy Novgorod',
}
name_series = pd.Series(data=list(name_map.values()), index=list(name_map.keys()))
name_series

Saturn Ramenskoe          Saturn REN TV Ramenskoe
Spartak Vladikavkaz    Spartak Alania Vladikavkaz
Genoa                                   Genoa CFC
FC Nizhniy Novgorod      FC Pari Nizhniy Novgorod
dtype: object

Next, we find the indices of all usages of the names we're purging in both datasets

In [12]:
purge_list = list(name_map.keys())

t_idx = t_clubs.index[t_clubs["name"].isin(purge_list)]
s_bundesliga_idx = s_bundesliga.index[s_bundesliga["club_name"].isin(purge_list)]
s_champ_idx = s_champ.index[s_champ["club_name"].isin(purge_list)]
s_eredivisie_idx = s_eredivisie.index[s_eredivisie["club_name"].isin(purge_list)]
s_liganos_idx = s_liganos.index[s_liganos["club_name"].isin(purge_list)]
s_ligue1_idx = s_ligue1.index[s_ligue1["club_name"].isin(purge_list)]
s_premierleague_idx = s_premierleague.index[s_premierleague["club_name"].isin(purge_list)]
s_premierliga_idx = s_premierliga.index[s_premierliga["club_name"].isin(purge_list)]
s_primeradiv_idx = s_primeradiv.index[s_primeradiv["club_name"].isin(purge_list)]
s_seriea_idx = s_seriea.index[s_seriea["club_name"].isin(purge_list)]

Using these indices, we extract the names that we need to update as `Series`

In [13]:
t_problems = t_clubs.loc[t_idx, "name"]
s_bundesliga_problems = s_bundesliga.loc[s_bundesliga_idx, "club_name"]
s_champ_problems = s_champ.loc[s_champ_idx, "club_name"]
s_eredivisie_problems = s_eredivisie.loc[s_eredivisie_idx, "club_name"]
s_liganos_problems = s_liganos.loc[s_liganos_idx, "club_name"]
s_ligue1_problems = s_ligue1.loc[s_ligue1_idx, "club_name"]
s_premierleague_problems = s_premierleague.loc[s_premierleague_idx, "club_name"]
s_premierliga_problems = s_premierliga.loc[s_premierliga_idx, "club_name"]
s_primeradiv_problems = s_primeradiv.loc[s_primeradiv_idx, "club_name"]
s_seriea_problems = s_seriea.loc[s_seriea_idx, "club_name"]

Next, we replace the problematic names in these `Series` with the appropriate counterparts by indexing `name_series` with them.

*The operations in the cell below replace the names correctly because `name_series` maps each name we wanted to purge to the we wanted to replace it with.*

In [14]:
t_fixed = name_series.loc[t_problems.values].values
s_bundesliga_fixed = name_series.loc[s_bundesliga_problems.values].values
s_champ_fixed = name_series.loc[s_champ_problems.values].values
s_eredivisie_fixed = name_series.loc[s_eredivisie_problems.values].values
s_liganos_fixed = name_series.loc[s_liganos_problems.values].values
s_ligue1_fixed = name_series.loc[s_ligue1_problems.values].values
s_premierleague_fixed = name_series.loc[s_premierleague_problems.values].values
s_premierliga_fixed = name_series.loc[s_premierliga_problems.values].values
s_primeradiv_fixed = name_series.loc[s_primeradiv_problems.values].values
s_seriea_fixed = name_series.loc[s_seriea_problems.values].values

Finally, we update the dataframes with the fixed names.

In [15]:
t_clubs.loc[t_idx, "name"] = t_fixed
s_bundesliga.loc[s_bundesliga_idx, "club_name"] = s_bundesliga_fixed
s_champ.loc[s_champ_idx, "club_name"] = s_champ_fixed
s_eredivisie.loc[s_eredivisie_idx, "club_name"] = s_eredivisie_fixed
s_liganos.loc[s_liganos_idx, "club_name"] = s_liganos_fixed
s_ligue1.loc[s_ligue1_idx, "club_name"] = s_ligue1_fixed
s_premierleague.loc[s_premierleague_idx, "club_name"] = s_premierleague_fixed
s_premierliga.loc[s_premierliga_idx, "club_name"] = s_premierliga_fixed
s_primeradiv.loc[s_primeradiv_idx, "club_name"] = s_primeradiv_fixed
s_seriea.loc[s_seriea_idx, "club_name"] = s_seriea_fixed

### Remove Unpaired Entities
In some cases, you may want to exclude entities that only appear in one dataset or the other from your database. Let's suppose that's the case here and remove all countries that only appear in one dataset or the other.

As before, we first extract the full list of countries found in both datasets and convert the lists to sets.

In [16]:
bundsliga_names = s_bundesliga["club_name"].unique()
champ_names = s_champ["club_name"].unique()
eredivisie_names = s_eredivisie["club_name"].unique()
liganos_names = s_liganos["club_name"].unique()
ligue1_names = s_ligue1["club_name"].unique()
premierleague_names = s_premierleague["club_name"].unique()
premierliga_names = s_premierliga["club_name"].unique()
primeradiv_names = s_primeradiv["club_name"].unique()
seria_names = s_seriea["club_name"].unique()

s_names = set(bundsliga_names.tolist() + champ_names.tolist() + eredivisie_names.tolist() + liganos_names.tolist() + ligue1_names.tolist() + premierleague_names.tolist() + premierliga_names.tolist() + primeradiv_names.tolist() + seria_names.tolist())

t_names = set(t_clubs["name"].unique().tolist())

Then, we convert these lists to sets and use set-difference operations to find the countries that only appear in one set.

In [17]:
t_diff = t_names.difference(s_names)
s_diff = s_names.difference(t_names)
total_diff = list(t_diff) + list(s_diff)

Finally, we drop all rows from both datasets that contain countries that match any of the countries in this list.

In [18]:
t_clubs = t_clubs.loc[~t_clubs["name"].isin(total_diff), :]
s_bundesliga = s_bundesliga.loc[~s_bundesliga["club_name"].isin(total_diff), :]
s_champ = s_champ.loc[~s_champ["club_name"].isin(total_diff), :]
s_eredivisie = s_eredivisie.loc[~s_eredivisie["club_name"].isin(total_diff), :]
s_liganos = s_liganos.loc[~s_liganos["club_name"].isin(total_diff), :]
s_ligue1 = s_ligue1.loc[~s_ligue1["club_name"].isin(total_diff), :]
s_premierleague = s_premierleague.loc[~s_premierleague["club_name"].isin(total_diff), :]
s_premierliga = s_premierliga.loc[~s_premierliga["club_name"].isin(total_diff), :]
s_primeradiv = s_primeradiv.loc[~s_primeradiv["club_name"].isin(total_diff), :]
s_seriea = s_seriea.loc[~s_seriea["club_name"].isin(total_diff), :]

## Replace Categorical Variables with Indicators

First, we create a `Series` that maps each country name to an integer.


In [19]:
combined_names = sorted(s_names | t_names)
name_codes = pd.Series(index=combined_names, data=np.arange(len(combined_names)))
name_codes

AC Ajaccio                       0
AC Cesena                        1
AC Fiorentina                    2
AC Horsens                       3
AC Le Havre                      4
                              ... 
Пресня Москва                  486
Торпедо Лужники Москва         487
ФК Элиста                      488
Факел Воронеж Воронеж          489
Энергия Текстильщик Камышин    490
Length: 491, dtype: int64

Next, we use this `Series` to map all countries in both datasets to the corresponding integers by indexing the `Series` with the names of the countries in the datasets.

In [20]:
t_clubs["name"] = name_codes.loc[t_clubs["name"]].values
s_bundesliga["club_name"] = name_codes.loc[s_bundesliga["club_name"]].values
s_champ["club_name"] = name_codes.loc[s_champ["club_name"]].values
s_eredivisie["club_name"] = name_codes.loc[s_eredivisie["club_name"]].values
s_liganos["club_name"] = name_codes.loc[s_liganos["club_name"]].values
s_ligue1["club_name"] = name_codes.loc[s_ligue1["club_name"]].values
s_premierleague["club_name"] = name_codes.loc[s_premierleague["club_name"]].values
s_premierliga["club_name"] = name_codes.loc[s_premierliga["club_name"]].values
s_primeradiv["club_name"] = name_codes.loc[s_primeradiv["club_name"]].values
s_seriea["club_name"] = name_codes.loc[s_seriea["club_name"]].values

<ipython-input-20-588390449f1a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_clubs["name"] = name_codes.loc[t_clubs["name"]].values
<ipython-input-20-588390449f1a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_bundesliga["club_name"] = name_codes.loc[s_bundesliga["club_name"]].values


Finally, we convert the columns to integer types.

In [21]:
t_clubs["name"] = t_clubs["name"].astype(np.int64)
s_bundesliga["club_name"] = s_bundesliga["club_name"].astype(np.int64)
s_champ["club_name"] = s_champ["club_name"].astype(np.int64)
s_eredivisie["club_name"] = s_eredivisie["club_name"].astype(np.int64)
s_liganos["club_name"] = s_liganos["club_name"].astype(np.int64)
s_ligue1["club_name"] = s_ligue1["club_name"].astype(np.int64)
s_premierleague["club_name"] = s_premierleague["club_name"].astype(np.int64)
s_premierliga["club_name"] = s_premierliga["club_name"].astype(np.int64)
s_primeradiv["club_name"] = s_primeradiv["club_name"].astype(np.int64)
s_seriea["club_name"] = s_seriea["club_name"].astype(np.int64)

<ipython-input-21-a3277c9ba177>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_clubs["name"] = t_clubs["name"].astype(np.int64)
<ipython-input-21-a3277c9ba177>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_bundesliga["club_name"] = s_bundesliga["club_name"].astype(np.int64)


## Find an Index

Before ingesting our data into the database, we need to find a unique index for each table

### Single-Column Index


In [22]:
len(t_clubs["name"].unique()) == len(t_clubs["name"])

True

### Multi-Column Index

When there's not an individual column that can act as an index, we search for combinations of columns that can make a unique index when combined.

To check a candidate set of columns:
1. Call `DataFrame.groupby()` on the list of candidate columns. *This creates a group of rows for each unique combination of candidate olumn values that appears in the Dataframe*
2. Call `GroupBy.size()` on the resulting grouped dataframe. *This counts the number of rows in each group*
3. Check whether every group has exactly 1 row.

We use this procedure below to check whether `country` and `year` can function as a joint index.

In [23]:
# Step 1: Group dataframe by candidate columns
grouped_le = le.groupby(["country", "year"])
# Step 2: Count rows in each group
counts = grouped_le.size()
print(counts)
# Step 3: Check whether every value equals 1
(counts == 1).all()

NameError: ignored

Great! This means no country-year pair appears more than once in the life expectancy table, so we can use `country` and `year` in combination as the index.

For the census dataset, you'd need every column to create a unique index (Check this for yourself). So we'll just plan to use the arbitrary, unique integers `census.index` as our table index.

## Export Data
After cleaning our datasets, resolving entity resolution problems, and choosing indices, we're ready to ingest our the data into our database.

In the past, students have found using Python for data ingestion slow and frustrating, so we won't populate the database here. Instead, we'll export both datasets and the country codes to CSVs. Then, we'll show you how to ingest these CSVs into your database using MySQL Workbench in the tutorial on data ingestion.


First, convert the `Series` of country codes to an equivalent `DataFrame`

In [24]:
name_codes = pd.DataFrame(data={'name': name_codes.index, 'code': name_codes.values})

Next, we use `DataFrame.to_csv` to write each dataset to a CSV file with a descriptive name.

*For `le` and `country_codes`, we set `index` to `False` because the indices of the `DataFrames` are meaningless integers that we don't need in our tables. For `census`, we rename the index and include it in the output because we decided  to use it as our table index, even though it's arbitrary.*

In [25]:
t_clubs.to_csv("t_clubs.csv", index=False)
t_players.to_csv("t_players.csv", index=False)
s_bundesliga.to_csv("s_bundesliga.csv", index=False)
s_champ.to_csv("s_champ.csv", index=False)
s_eredivisie.to_csv("s_eredivisie.csv", index=False)
s_liganos.to_csv("s_liganos.csv", index=False)
s_ligue1.to_csv("s_ligue1.csv", index=False)
s_premierleague.to_csv("s_premierleague.csv", index=False)
s_premierliga.to_csv("s_premierliga.csv", index=False)
s_primeradiv.to_csv("s_primeradiv.csv", index=False)
s_seriea.to_csv("s_seriea.csv", index=False)

name_codes.to_csv("name_codes.csv", index=False)


Finally, we download these files to our local machine, so we can put them into MySQL Workbench later.

In [28]:
from google.colab import files
files.download("t_clubs.csv")
files.download("t_players.csv")
files.download("s_bundesliga.csv")
files.download("s_champ.csv")
files.download("s_eredivisie.csv")
files.download("s_liganos.csv")
files.download("s_ligue1.csv")
files.download("s_premierleague.csv")
files.download("s_premierliga.csv")
files.download("s_primeradiv.csv")
files.download("s_seriea.csv")

files.download("name_codes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Exercises
Check out [these exercises](https://drive.google.com/open?id=1kjLaYC_KJUlltm-iAzgF5VmHgb7Uer0z) to practice the processing techniques you learned above!